<a href="https://colab.research.google.com/github/HernanJMaurice/androidSensors/blob/main/HPC/Maurice_Hernan_Ejercicio_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

url = "https://fcm.googleapis.com/fcm/send"

data = {
    "to":"/topics/all",
    "notification":{
        "sound": "default",
        "body" : "Este es el cuerpo",
        "title": "Este es el Titulo",
        "content_available" : True,
        "priority" : "high"
    },
    "data":{
        "sound": "default",
        "body" :"Este es el cuerpo",
        "title" :"Este es el Titulo",
        "content_available": True,
        "priority" :"high"
    }
}

headers = {'Authorization': 'key=AAAACPWgRFU:APA91bEK8Eo-3Mdwr_hzjj-rVv4D4KAUPBzCA0Uot_UdkOqL_zR2Op8M5gYNX3txi7tWotiudXuBtZtULhgai0rZmdSHEiAGw4ZorFSq37nPupC1wS8ckti3UYrN3wpvby9ljSPCq-Rq', 'Content-Type': 'application/json'}
data=json.dumps(data)
r = requests.post(url, data, headers=headers)
print(r.content)


In [174]:
pip install requests


In [255]:
code = """

#include <string.h>
#include <stdio.h>
#include <mpi.h>
#include<stdlib.h>

int main(int argc, char** argv)
{
  int myRank, numProcs;
  int tag=1;
  int lenght = atoi(argv[3]);
  char mensaje[4];
  char mensajeRespuesta[100];
  char respuesta[lenght];
  MPI_Status status;

  MPI_Init(NULL, NULL);

  MPI_Comm_rank( MPI_COMM_WORLD, &myRank );
  MPI_Comm_size( MPI_COMM_WORLD, &numProcs );

  //Si No es el primer proceso Ejecuta
  if( myRank != 0 )
  {
      int dest = 0;
      MPI_Recv(&mensaje, 4, MPI_CHAR, 0, tag, MPI_COMM_WORLD, &status);

      if(mensaje[0] == mensaje[1])
      {

          sprintf( mensajeRespuesta, "|" );

      }else if( mensaje[0] == ' ' || mensaje[1]== ' ')
      {
          sprintf( mensajeRespuesta, " " );
      }else
      {
          sprintf( mensajeRespuesta, "*" );
      }


      //sprintf( mensajeRespuesta, "Recibido %d y %d del proceso %d de %d", mensaje[0],  mensaje[1], myRank, numProcs );	
      MPI_Send(&mensajeRespuesta, strlen(mensajeRespuesta), MPI_CHAR, dest, tag, MPI_COMM_WORLD);

  }
  else
  {
    
    for(int i=1; i<numProcs; i++)
    {
        strcpy(mensaje, "");
        if(i < strlen(argv[1]) && i < strlen(argv[2]))
        {  
            sprintf( mensaje, "%c%c", argv[1][i-1], argv[2][i-1] );	
        }else if( i <= strlen(argv[1]) && i > strlen(argv[2]) )
        {
            sprintf( mensaje, "%c ", argv[1][i-1]);	
        }else if( i > strlen(argv[1]) && i <= strlen(argv[2]) )
        {
            sprintf( mensaje, " %c", argv[2][i-1]);	
        }

        MPI_Send(mensaje, 2, MPI_CHAR, i, tag, MPI_COMM_WORLD);
        
        printf("Enviado %s a %d\\n",mensaje, i);
    }

    strcpy(respuesta,"");

    for(int i=1; i<numProcs; i++)
    {
       MPI_Recv(&mensajeRespuesta, 100, MPI_CHAR, i, tag, MPI_COMM_WORLD, &status);
       //printf( "%s\\n", mensajeRespuesta);
       //printf( "Llego %s del proceso %d\\n", respuesta, i );
       strcat(respuesta, mensajeRespuesta);
    }
    
    printf( "-%s\\n",argv[1]);
    printf( "-%s-\\n", respuesta );
    printf( "-%s\\n",argv[2]);

  }

  MPI_Finalize();
}
"""
text_file = open("SecuenciacionADN.c", "w")
text_file.write(code)
text_file.close()

In [256]:
!mpicc -o SecuenciacionADN SecuenciacionADN.c 

In [187]:
try:
    primer_Cadena_ADN = 'ACTGAGTCGGATGACTGAGTCGGATG'#@param {type: "string"}
    segunda_Cadena_ADN = 'ACTGAGTCGGAA GGGAGTCG ATGGGTTAG  CG TA '#@param {type: "string"}

    import re #regex
    import json 
    import requests

    # Chequeo que solo tenga Nucleotidos permitidos.
    if not primer_Cadena_ADN or not segunda_Cadena_ADN:
        raise Exception("Cadena de ADN Vacia.")
        
    # Lo paso siempre a Mayuscula.
    primer_Cadena_ADN = primer_Cadena_ADN.upper()
    segunda_Cadena_ADN = segunda_Cadena_ADN.upper()

    pattern = re.compile("(A|C|T|G| )+")

    # Chequeo que solo tenga Nucleotidos permitidos.
    if pattern.fullmatch(primer_Cadena_ADN) is None or pattern.fullmatch(segunda_Cadena_ADN) is None :
            raise Exception("Cadena Incorrecta: Los nucleotidos pueden ser: A, C, T, G o ' '.")


    if len(primer_Cadena_ADN) > len(segunda_Cadena_ADN) :
            largo = len(primer_Cadena_ADN)
    else :
            largo = len(segunda_Cadena_ADN)

    mensaje = "Cadenas:", f'"{primer_Cadena_ADN}"', "y", f'"{segunda_Cadena_ADN}"'

    print('!mpirun --allow-run-as-root -n',largo+1, 'SecuenciacionADN', f'"{primer_Cadena_ADN}"', f'"{segunda_Cadena_ADN}"', largo)


    url = "https://fcm.googleapis.com/fcm/send"

    data = {
        "to":"/topics/all",
        "notification":{
            "sound": "default",
            "body" : mensaje,
            "title": "Ejecucion Lista",
            "content_available" : True,
            "priority" : "high"
        }
    }

    headers = {'Authorization': 'key=AAAACPWgRFU:APA91bEK8Eo-3Mdwr_hzjj-rVv4D4KAUPBzCA0Uot_UdkOqL_zR2Op8M5gYNX3txi7tWotiudXuBtZtULhgai0rZmdSHEiAGw4ZorFSq37nPupC1wS8ckti3UYrN3wpvby9ljSPCq-Rq', 'Content-Type': 'application/json'}
    data=json.dumps(data)
    r = requests.post(url, data, headers=headers)
    if (r.status_code == 200) :

        print("Notificacion enviada exitosamente.")
    else :
        print("Error en la Notificacion.")
    


except Exception as e:
          print('Error: ', e.args )

!mpirun --allow-run-as-root -n 40 SecuenciacionADN "ACTGAGTCGGATGACTGAGTCGGATG" "ACTGAGTCGGAA GGGAGTCG ATGGGTTAG  CG TA " 39
Notificacion enviada exitosamente.


In [257]:
!mpirun --allow-run-as-root -n 39 SecuenciacionADN "ACTGAGTCGGATGACTGAGTCGGATG" "ACTGAGTCGGAA GGGAGTCG ATGGGTTAG  CG TA" 38

Enviado AA a 1
Enviado CC a 2
Enviado TT a 3
Enviado GG a 4
Enviado AA a 5
Enviado GG a 6
Enviado TT a 7
Enviado CC a 8
Enviado GG a 9
Enviado GG a 10
Enviado AA a 11
Enviado TA a 12
Enviado G  a 13
Enviado AG a 14
Enviado CG a 15
Enviado TG a 16
Enviado GA a 17
Enviado AG a 18
Enviado GT a 19
Enviado TC a 20
Enviado CG a 21
Enviado G  a 22
Enviado GA a 23
Enviado AT a 24
Enviado TG a 25
Enviado  a 26
Enviado  G a 27
Enviado  T a 28
Enviado  T a 29
Enviado  A a 30
Enviado  G a 31
Enviado    a 32
Enviado    a 33
Enviado  C a 34
Enviado  G a 35
Enviado    a 36
Enviado  T a 37
Enviado  A a 38
-ACTGAGTCGGATGACTGAGTCGGATG
-|||||||||||* ******** ****     ||  |  -
-ACTGAGTCGGAA GGGAGTCG ATGGGTTAG  CG TA
